# SentenceTransformer fine-tuning (binary classification)
Fine-tune a sentence-transformer for the synthetic binary labels. Supports freezing the encoder to only train the classifier head; extendable to LoRA if desired.


In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, losses, evaluation, InputExample
from torch.utils.data import DataLoader


/var/git/genai-project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Params
DATA_PATH = Path('../data/synthetic_normal_2_labeled.csv')
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
TEST_SIZE = 0.2
VAL_SIZE = 0.1
BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 0.01
FREEZE_ENCODER = False
SEED = 42
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {DEVICE}')


Using device: cuda


In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


In [ ]:
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Missing dataset at {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
df['sentence'] = df['sentence'].astype(str)
df['label'] = df['label'].astype(int)
print(df['label'].value_counts(normalize=True))


label
1    0.500008
0    0.499992
Name: proportion, dtype: float64


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['sentence'], df['label'], test_size=TEST_SIZE, random_state=SEED, stratify=df['label']
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=VAL_SIZE, random_state=SEED, stratify=y_train
)
print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")


Train: 45968, Val: 5108, Test: 12769


In [ ]:
train_examples = [InputExample(texts=[s], label=int(l)) for s, l in zip(X_train, y_train)]
val_examples = [InputExample(texts=[s], label=int(l)) for s, l in zip(X_val, y_val)]
test_examples = [InputExample(texts=[s], label=int(l)) for s, l in zip(X_test, y_test)]


In [ ]:
model = SentenceTransformer(MODEL_NAME, device=DEVICE)

if FREEZE_ENCODER:
    transformer = model[0]
    for param in transformer.auto_model.parameters():
        param.requires_grad = False
    print('Encoder frozen; only classifier head will be trained')


In [ ]:
def collate_fn(batch):
    sentences = [ex.texts[0] if isinstance(ex.texts, (list, tuple)) else ex.texts for ex in batch]
    labels = torch.tensor([int(ex.label) for ex in batch], dtype=torch.long)
    features = model.tokenize(sentences)
    return features, labels

train_dataloader = DataLoader(
    train_examples,
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
)

val_sentences = list(X_val)
val_labels = list(map(int, y_val))


In [ ]:
embed_dim = model.get_sentence_embedding_dimension()
classifier = torch.nn.Linear(embed_dim, 2).to(DEVICE)
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import torch.nn.functional as F

optimizer = AdamW(list(model.parameters()) + list(classifier.parameters()), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

def forward_batch(features):
    features = {k: v.to(DEVICE) for k, v in features.items()}
    emb = model(features)['sentence_embedding']
    logits = classifier(emb)
    return logits

def eval_split(sentences, labels):
    model.eval()
    classifier.eval()
    all_logits = []
    with torch.no_grad():
        for i in range(0, len(sentences), BATCH_SIZE):
            batch_sents = sentences[i:i+BATCH_SIZE]
            feats = model.tokenize(batch_sents)
            logits = forward_batch(feats)
            all_logits.append(logits.cpu())
    logits = torch.cat(all_logits)
    preds = logits.argmax(dim=1).numpy()
    acc = accuracy_score(labels, preds)
    bal = balanced_accuracy_score(labels, preds)
    return acc, bal

for epoch in range(EPOCHS):
    model.train()
    classifier.train()
    total_loss = 0.0
    for features, labels in train_dataloader:
        labels = labels.to(DEVICE)
        logits = forward_batch(features)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    train_acc, train_bal = eval_split(list(X_train), list(map(int, y_train)))
    val_acc, val_bal = eval_split(val_sentences, val_labels)
    print(f"Epoch {epoch+1}/{EPOCHS} - loss {total_loss/len(train_dataloader):.4f} "
          f"train_acc {train_acc:.3f} train_bal {train_bal:.3f} "
          f"val_acc {val_acc:.3f} val_bal {val_bal:.3f}")


Epoch 1/2 - loss 0.2924 train_acc 0.938 train_bal 0.938 val_acc 0.898 val_bal 0.898
Epoch 2/2 - loss 0.1722 train_acc 0.966 train_bal 0.966 val_acc 0.903 val_bal 0.903


In [ ]:
from sklearn.metrics import classification_report, balanced_accuracy_score

test_sentences = list(X_test)
test_labels = list(map(int, y_test))
model.eval(); classifier.eval()
all_logits = []
with torch.no_grad():
    for i in range(0, len(test_sentences), BATCH_SIZE):
        batch_sents = test_sentences[i:i+BATCH_SIZE]
        feats = model.tokenize(batch_sents)
        logits = forward_batch(feats)
        all_logits.append(logits.cpu())
logits = torch.cat(all_logits)
probs = torch.softmax(logits, dim=1)
preds = probs.argmax(dim=1).numpy()

print('Balanced accuracy:', balanced_accuracy_score(test_labels, preds))
print('\nClassification report:\n')
print(classification_report(test_labels, preds, digits=3))


Balanced accuracy: 0.9125235514392873

Classification report:

              precision    recall  f1-score   support

           0      0.902     0.926     0.914      6384
           1      0.924     0.899     0.911      6385

    accuracy                          0.913     12769
   macro avg      0.913     0.913     0.913     12769
weighted avg      0.913     0.913     0.913     12769

